<a href="https://colab.research.google.com/github/invinciblesohail/Pract/blob/master/HyperParametersTuning_TPSSep2021_(HyperOpt).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HyperParametersTuning TPSSep2021 (HyperOpt)

Before starting, two libraries optuna and scikit-optimize for optimize must be installed

In [18]:
!pip install scikit-optimize

In [19]:
! pip install optuna

In [20]:
from numbers import Real
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import pipeline
from functools import partial
import optuna
from skopt import space
from skopt import gp_minimize 
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import gc

## Setting up Notebook for kaggle environment to download data

In [ ]:
! pip install kaggle

In [23]:
! mkdir ~/.kaggle

In [24]:
! cp kaggle.json ~/.kaggle/

In [25]:
! chmod 600 ~/.kaggle/kaggle.json

In [26]:
! kaggle competitions download tabular-playground-series-sep-2021

 92% 186M/203M [00:02<00:00, 98.1MB/s]
100% 203M/203M [00:02<00:00, 84.8MB/s]
  0% 0.00/1.10M [00:00<?, ?B/s]
100% 1.10M/1.10M [00:00<00:00, 155MB/s]
 98% 386M/394M [00:02<00:00, 156MB/s]
100% 394M/394M [00:02<00:00, 165MB/s]


## Data donwloaded successfully

In [27]:
!ls

kaggle.json  sample_data  sample_solution.csv.zip  test.csv.zip  train.csv.zip


### Now need to unzip the file

In [28]:
! unzip train.csv.zip

Archive:  train.csv.zip
  inflating: train.csv               


In [29]:
! unzip test.csv.zip

Archive:  test.csv.zip
  inflating: test.csv                


In [30]:
! unzip sample_solution.csv.zip

Archive:  sample_solution.csv.zip
  inflating: sample_solution.csv     


In [31]:
train = pd.read_csv('train.csv')
X_test = pd.read_csv("test.csv")
sample_solution = pd.read_csv("sample_solution.csv")

In [32]:
train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300,8.6331,4.0607,26.8670,-1.180,10961.00,1.5397,135.3200,-1.49650,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.0,...,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.3980,-2.2012,6871.0,3.8862,-0.00558,5252.100,166.690,1.60740,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.471,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284,6.1872,1.0419,4.6404,31.877,123620.00,1.3951,125.8100,1.19890,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.5,...,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.0510,205.6900,4295.3,13.3880,0.46843,754.610,83.233,1.18900,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.720,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.873,7.5463,1.9967,1.9526,817.760,-2948.70,2.0054,1.6826,1.19680,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.4,...,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.3731,391.3700,2965.3,NaN,0.49459,43.524,138.520,1.10790,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.486,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.840,7.8490,1.0379,8.0030,12.349,-195.28,2.5598,92.1420,0.63789,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.5,...,0.372830,1.5606,1.835400e+10,-3.4298,6485700.0,2120.000,3.081200e+10,34.0560,157.4300,3724.5,8.4211,0.40778,2971.200,204.700,-0.97998,9.94050,12011.000,0.99898,5.063400e+13,1.2261,0.250200,0.72974,373690.0,194.650,120.930,0.260710,0.234240,-0.002794,1.442300e+09,-0.01182,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.900,7.2783,3.9757,NaN,NaN,29520.00,3.4225,96.7250,0.79725,215.570,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.97413,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.1,...,0.085690,1.5846,3.825200e+10,130.7000,102100.0,1951.800,1.142800e+10,58.5660,176.8300,1279.0,4.9662,0.47912,-70.278,10.887,1.14340,6.19120,197.470,NaN,1.574800e+13,1.0083,0.339530,13.48600,201300.0,38.842,324.000,0.238250,0.141550,0.002208,5.830700e+09,0.92739,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [33]:
X_test.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
0,957919,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.000,1.17200,5.175000e+09,0.25831,3.42050,0.044983,1001.600,10.1040,10.5360,19.517,-1.1209,1577.60,2.18470,96.109,1.1535,95.865,2.847100e+12,9.642100e+10,6406000.0,0.002990,6.6353,-0.015913,21723.0,67604.0,-0.000227,6.046200e+16,203.580,301.790,1.3461,5524.60,...,NaN,0.510410,4.0322,2.477900e+11,69.2640,5494800.0,1239.00,5.349300e+08,5.3086,182.590,829.17,5.2319,0.010301,3640.800,112.120,-0.43442,11.16100,3755.700,0.99926,5.338200e+10,1.3867,-0.001878,58.7940,75657.0,187.240,1258.90,0.163340,0.055398,0.020002,7.956400e+08,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357
1,957920,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.430,1.89790,4.905100e+09,0.28471,2.13870,-0.034328,35.223,6.5873,12.7340,29.590,4.1639,31915.00,3.11580,109.430,-0.8522,991.370,4.700100e+09,1.618200e+12,967810.0,1.021500,2.9617,0.810310,415570.0,384720.0,0.002243,6.712900e+15,228.590,33.952,1.4938,2067.50,...,-5.3544,0.067286,5.4044,2.462600e+11,810.3000,860670.0,2165.20,2.711500e+08,10.2320,515.410,262.71,5.6357,0.374220,420.160,116.230,1.26980,0.78619,1207.000,1.00070,2.065000e+13,1.2888,0.248850,9.9857,16323.0,244.460,647.58,0.253820,0.008800,0.018479,1.120600e+08,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125
2,957921,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.810,-0.48404,3.800000e+06,0.37173,0.40245,0.015119,1991.600,6.0050,0.9919,11.949,888.0700,887.70,3.52370,108.140,-1.6542,528.140,2.463600e+12,9.005900e+09,7259.3,1.007400,2.6123,1.062500,-208010.0,65708.0,0.001391,8.129800e+13,-84.032,NaN,1.7556,4729.20,...,-1.4805,0.064203,3.0197,1.684400e+11,212.8300,7891000.0,277.16,4.047900e+10,1.1170,393.510,8859.10,8.8098,0.405350,393.650,305.140,1.32660,30.85200,2803.700,0.99925,4.722600e+12,1.5003,0.493380,37.0470,139070.0,2464.000,4724.80,0.299160,0.093046,0.018516,6.519300e+07,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797
3,957922,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,-26.473,-2.12930,4.675400e+09,0.26659,1.09680,-0.083571,1987.500,6.8989,11.1230,23.164,133.5700,-815.32,2.72950,82.011,3.8374,304.570,-1.315900e+10,-8.324500e+09,2297700.0,0.099522,4.9249,0.659900,48773.0,199560.0,-0.000621,3.993800e+16,1400.400,503.660,1.9131,2811.60,...,-2.3828,0.128580,1.4494,2.681000e+11,66.5990,-733900.0,843.42,1.216700e+10,3.6871,243.590,1199.70,4.5092,0.377030,-36.886,11.401,0.97205,23.64200,1882.700,0.99922,1.568000e+13,1.0456,0.548180,27.6650,813040.0,1332.000,359.17,0.026553,0.668620,-0.001260,1.995100e+09,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910
4,957923,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,5999.400,1.87380,1.857300e+09,0.32259,2.21150,-0.107200,661.070,6.5391,6.8495,7.258,468.7400,10316.00,0.79638,111.520,1.4098,429.790,4.121700e+12,-3.945700e+08,7119.4,1.027700,16.1210,0.994510,372800.0,12042.0,-0.000659,1.869200e+14,-129.580,933.150,2.7392,490.84,...,-3.2187,0.502290,6.9219,1.487300e+11,9.9686,-35362.0,854.84,2.603900e+10,13.9430,14.552,675.62,6.8424,0.376850,3452.100,35.434,0.94128,0.95142,-56.004,0.99969,2.009700e+13,1.2255,0.713930,14.1820,4362.6,12.007,10653.00,0.165300,0.227090,0.020485,-1.050400e+07,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796


In [34]:
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
count,957919.000000,942672.000000,942729.000000,942428.000000,942359.000000,942514.000000,942398.000000,942415.000000,9.425460e+05,9.426700e+05,942696.000000,942494.000000,9.423260e+05,942455.000000,942697.000000,942410.000000,942475.000000,942492.000000,942594.000000,942445.000000,942464.000000,942465.000000,942641.000000,942563.000000,942289.000000,942413.000000,9.425610e+05,9.424750e+05,9.426540e+05,942504.000000,942527.000000,942241.000000,9.423900e+05,9.424270e+05,942671.000000,9.425850e+05,942556.000000,942607.000000,942485.000000,942360.000000,...,942599.000000,942573.000000,9.424340e+05,942292.000000,9.425340e+05,942470.000000,9.423960e+05,942603.000000,942371.000000,942474.000000,942437.000000,942412.000000,942427.000000,942462.000000,942505.000000,942320.000000,942634.000000,942654.000000,9.426310e+05,942485.000000,942393.000000,942570.000000,9.427510e+05,942319.000000,942721.000000,942533.000000,942375.000000,942535.000000,9.425790e+05,942390.000000,942554.000000,942420.000000,942509.000000,942686.000000,942481.000000,942360.000000,9.423300e+05,942512.000000,942707.000000,957919.000000
mean,478959.000000,0.090201,0.345964,4068.744207,0.201214,0.304869,-0.071458,1620.843815,3.771642e+05,1.806054e+15,5323.442367,0.729518,1.844436e+09,0.247759,6.999152,0.019394,444.804439,6.892520,4.491982,22.448660,203.782031,61070.567499,2.269597,87.166891,0.341240,414.983986,3.379988e+12,1.254164e+12,2.256887e+06,0.329054,7.882455,0.394211,1.344700e+05,3.579055e+05,-0.000005,2.782318e+16,185.485192,406.072571,1.768743,1981.074344,...,0.194237,3.240370,1.053811e+11,152.850973,6.128694e+06,635.301741,3.251414e+10,26.598567,207.260994,3805.100719,6.733556,0.366754,4868.608259,132.264261,0.821190,13.120677,3849.687894,0.999970,1.415684e+13,1.683151,0.425840,20.213776,3.215760e+05,548.647493,3855.616303,0.178027,0.160832,0.014196,1.671873e+09,0.398622,-19.926398,2.074530,23.885245,1.748777,63152.973540,1.208876,4.276905e+16,3959.204669,0.559267,0.498492
std,276527.540591,0.043564,0.146251,6415.829440,0.212510,0.145343,2.123777,1276.281403,3.454325e+05,2.335204e+15,10068.380032,1.495535,2.125478e+09,0.101156,6.621472,0.101880,631.327086,1.715735,3.899241,14.609968,281.015052,89859.616863,0.896001,37.350264,1.643595,314.822410,5.655524e+12,1.642736e+12,2.303909e+06,0.433861,5.939864,0.781932,2.036605e+05,4.626670e+05,0.001534,3.452707e+16,701.769083,314.708639,0.588984,1958.688237,...,0.162364,1.992068,9.896094e+10,227.888751,8.767985e+06,583.468704,3.069150e+10,25.454605,158.215276,3532.626437,3.158678,0.146413,8428.424099,110.058170,0.712584,12.740238,6437.386650,0.001535,1.640385e+13,0.712016,0.283651,19.617660,3.277370e+05,864.175628,6670.256245,0.123319,0.141616,0.016259,1.875733e+09,0.298071,18.578439,0.895793,45.581360,10.088848,92435.016241,0.114959,6.732441e+16,3155.991777,0.408426,0.499998
min,0.000000,-0.149910,-0.019044,-9421.700000,-0.082122,-0.006990,-12.791000,-224.800000,-2.984300e+04,-1.153300e+15,-26404.000000,-8.086300,-4.081000e+08,-0.103800,-0.853760,-0.335660,-116.880000,-3.664500,-0.066527,-4.422500,-58.834000,-84079.000000,-6.009400,-20.514000,-5.735200,-71.502000,-6.956700e+11,-9.384200e+11,-4.706000e+05,-0.005659,-0.529990,-3.813500,-3.496500e+05,-6.055900e+05,-0.003881,-2.068900e+16,-2414.300000,-40.881000,0.546100,-433.700000,...,-0.017615,0.964200,-7.345700e+10,-28.752000,-2.992000e+06,-74.545000,-5.949500e+09,-7.616400,-22.576000,-296.780000,-0.257570,-0.012238,-12829.000000,-12.922000,-3.293300,-1.352400,-7764.300000,0.996100,-5.714600e+12,0.608200,-0.034559,-4.294900,-2.277700e+05,-222.210000,-11581.000000,-0.029027,-0.066726,-0.007535,-5.877000e+08,-0.042355,-105.860000,0.27704

In [35]:
X_test.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118
count,4.934740e+05,485662.000000,485583.000000,485679.000000,485741.000000,485597.000000,485561.000000,485541.000000,4.856190e+05,4.856560e+05,485578.000000,485553.000000,4.853870e+05,485543.000000,485724.000000,485434.000000,485534.000000,485489.000000,485469.000000,485486.000000,485654.000000,485473.000000,485581.000000,485566.000000,485710.000000,485595.000000,4.853540e+05,4.855110e+05,4.855220e+05,485530.000000,485566.000000,485640.000000,4.856130e+05,4.856650e+05,485557.000000,4.855710e+05,485491.000000,485489.000000,485607.000000,485497.000000,...,485480.000000,485513.000000,485687.000000,4.855690e+05,485502.000000,4.855440e+05,485644.000000,4.855270e+05,485609.000000,485516.000000,485532.000000,485597.000000,485663.000000,485501.000000,485449.000000,485530.000000,485567.000000,485628.000000,485689.000000,4.855010e+05,485336.000000,485491.000000,485547.000000,4.856860e+05,485433.000000,485446.000000,485585.000000,485487.000000,485531.000000,4.855120e+05,485461.000000,485701.000000,485585.000000,485449.000000,485718.000000,485532.000000,485497.000000,4.853910e+05,485711.000000,485589.000000
mean,1.204656e+06,0.090191,0.346143,4047.202400,0.201503,0.304879,-0.071511,1617.036351,3.767998e+05,1.808500e+15,5331.081920,0.727481,1.842218e+09,0.247824,6.980486,0.019480,444.993705,6.887259,4.492907,22.405279,204.179272,61061.622910,2.268453,87.134912,0.346086,415.173762,3.365555e+12,1.254414e+12,2.255305e+06,0.328629,7.884026,0.395991,1.345694e+05,3.585715e+05,-0.000006,2.790835e+16,184.757012,406.837380,1.766992,1979.060918,...,1.563396,0.194331,3.237435,1.055740e+11,153.404052,6.138670e+06,634.975960,3.246176e+10,26.556542,207.341037,3809.778817,6.738272,0.366732,4863.773239,132.241696,0.821535,13.101304,3836.738568,0.999967,1.419011e+13,1.682167,0.426163,20.229069,3.216035e+05,550.181750,3884.011079,0.177768,0.160800,0.014241,1.674677e+09,0.398130,-19.840970,2.075146,23.846392,1.762625,63051.287541,1.209049,4.284258e+16,3968.585512,0.558734
std,1.424538e+05,0.043538,0.145889,6383.224961,0.212553,0.145435,2.125074,1272.769177,3.450146e+05,2.337721e+15,10073.656008,1.496929,2.123731e+09,0.101137,6.618522,0.101860,630.937562,1.717740,3.886045,14.595998,281.270488,89874.183727,0.893400,37.433377,1.641953,315.164640,5.636314e+12,1.642750e+12,2.304907e+06,0.433810,5.940837,0.780796,2.036919e+05,4.629596e+05,0.001535,3.460216e+16,700.217260,314.686415,0.587023,1955.656320,...,9.069221,0.162469,1.989798,9.885584e+10,228.408166,8.762835e+06,583.673924,3.064174e+10,25.460740,158.195120,3528.045827,3.161756,0.146598,8410.948639,110.020804,0.713190,12.720450,6410.389934,0.001534,1.642669e+13,0.710438,0.283865,19.667942,3.278863e+05,865.594215,6709.041960,0.123451,0.141757,0.016273,1.877023e+09,0.297971,18.513063,0.895702,45.544851,10.096886,92279.031878,0.115074,6.743966e+16,3161.273268,0.408472
min,9.579190e+05,-0.153190,-0.019060,-9436.300000,-0.080716,-0.007007,-12.841000,-215.660000,-2.780900e+04,-1.254900e+15,-26331.000000,-8.028200,-4.060200e+08,-0.101720,-0.973450,-0.338470,-105.630000,-3.740300,-0.066620,-4.092900,-56.885000,-90334.000000,-5.872000,-21.487000,-5.653200,-72.167000,-7.015000e+11,-9.265200e+11,-4.719000e+05,-0.005599,-0.503680,-3.689900,-3.297800e+05,-6.373800e+05,-0.003885,-2.126100e+16,-2359.000000,-39.572000,0.573180,-394.330000,...,-24.362000,-0.016956,0.963910,-6.521800e+10,-29.707000,-2.728400e+06,-73.071000,-5.937200e+09,-6.277800,-22.567000,-288.040000,-0.254210,-0.012285,-11655.000000,-12.819000,-3.174200,-1.305200,-7890.300000,0.996050,-4.334700e+12,0.650700,-0.036089,-3.923800,-2.645700e+05,-223.190000,-11039.000000,-0.028992,-0.049736,-0.007125,-5.517100e+08,-0.042181,-104.7400

## Deviding dependent and independent variables and adding new features¶
From the count rows in the tables above, we see that columns like "Id" are unique, hence wont contribute for our predictions. Hence we remove them.

In [36]:
y = train['claim']
train.pop('claim')
X_test.pop('id')
X=train
X['sum_na'] = X.isna().sum(axis=1)
X_test['sum_na'] = X_test.isna().sum(axis=1)
del train
gc.collect()

747

In [37]:
X.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,sum_na
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,86.489,0.59881,1.423200e+09,0.27240,9.455600,-0.050305,1938.300,8.6331,4.0607,26.8670,-1.180,10961.00,1.5397,135.3200,-1.49650,440.080,2.590100e+12,2.194200e+09,2968800.0,0.001431,13.3270,0.75050,18509.0,146820.0,-0.000276,1.090600e+16,1705.400,414.29,3.5392,1888.0,...,-0.001081,6.1244,1.231800e+11,275.9200,5308500.0,1704.000,5.022400e+10,53.3980,-2.2012,6871.0,3.8862,-0.00558,5252.100,166.690,1.60740,0.66534,7768.900,0.99662,1.125700e+11,2.2432,0.934160,0.65056,94569.0,21.471,8214.100,0.288010,0.097826,0.001071,1.412400e+09,0.11093,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,9953.600,1.20930,3.334100e+09,0.28631,-0.012858,-0.019912,10.284,6.1872,1.0419,4.6404,31.877,123620.00,1.3951,125.8100,1.19890,136.450,9.098100e+09,4.004100e+10,1564000.0,0.000204,3.1074,1.50330,238000.0,21440.0,-0.001344,3.079400e+16,229.100,844.82,1.4680,4726.5,...,0.254100,6.9191,1.832400e+11,9.6510,32800.0,1480.600,2.300600e+10,44.0510,205.6900,4295.3,13.3880,0.46843,754.610,83.233,1.18900,29.55000,7343.700,0.99815,4.877700e+13,1.2708,-0.000969,5.29520,6779.0,227.720,34.342,0.340300,0.143370,0.049276,1.903200e+09,0.97673,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,15827.000,0.38164,1.230300e+09,0.25807,2.455600,NaN,26.873,7.5463,1.9967,1.9526,817.760,-2948.70,2.0054,1.6826,1.19680,74.624,-3.273900e+10,5.718900e+10,11058.0,-0.003097,8.0241,1.13180,27940.0,862460.0,-0.002207,5.849100e+13,-897.840,NaN,1.3561,3063.4,...,0.260260,6.1052,1.013300e+11,357.2700,1476600.0,90.845,1.306200e+09,2.3731,391.3700,2965.3,NaN,0.49459,43.524,138.520,1.10790,0.91948,47.915,NaN,1.510500e+12,3.4663,0.560950,4.13090,95531.0,39.486,-83.148,0.084881,0.032222,0.001668,1.436500e+07,0.20102,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,5
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,-36.837,1.10960,1.223100e+09,0.30944,10.370000,-0.106260,533.840,7.8490,1.0379,8.0030,12.349,-195.28,2.5598,92.1420,0.63789,1054.900,-1.204100e+10,5.187300e+12,1475400.0,1.036500,1.1903,0.98941,301200.0,NaN,-0.000007,-9.299200e+13,-10.818,1020.30,2.9553,3342.5,...,0.372830,1.5606,1.835400e+10,-3.4298,6485700.0,2120.000,3.081200e+10,34.0560,157.4300,3724.5,8.4211,0.40778,2971.200,204.700,-0.97998,9.94050,12011.000,0.99898,5.063400e+13,1.2261,0.250200,0.72974,373690.0,194.650,120.930,0.260710,0.234240,-0.002794,1.442300e+09,-0.01182,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,2
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,144.120,1.05310,2.634100e+09,0.29782,2.654800,NaN,1808.900,7.2783,3.9757,NaN,NaN,29520.00,3.4225,96.7250,0.79725,215.570,1.732600e+13,2.635200e+12,2161200.0,0.895470,6.8257,0.97413,142620.0,231350.0,0.001257,1.012500e+16,51.508,293.76,1.3351,3042.1,...,0.085690,1.5846,3.825200e+10,130.7000,102100.0,1951.800,1.142800e+10,58.5660,176.8300,1279.0,4.9662,0.47912,-70.278,10.887,1.14340,6.19120,197.470,NaN,1.574800e+13,1.0083,0.339530,13.48600,201300.0,38.842,324.000,0.238250,0.141550,0.002208,5.830700e+09,0.92739,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,8


In [38]:
X_test.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,...,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,sum_na
0,0.165850,0.487050,1295.00,0.02310,0.31900,0.90188,573.29,3743.7,2.705700e+12,6221.000,1.17200,5.175000e+09,0.25831,3.42050,0.044983,1001.600,10.1040,10.5360,19.517,-1.1209,1577.60,2.18470,96.109,1.1535,95.865,2.847100e+12,9.642100e+10,6406000.0,0.002990,6.6353,-0.015913,21723.0,67604.0,-0.000227,6.046200e+16,203.580,301.790,1.3461,5524.60,-0.003163,...,0.510410,4.0322,2.477900e+11,69.2640,5494800.0,1239.00,5.349300e+08,5.3086,182.590,829.17,5.2319,0.010301,3640.800,112.120,-0.43442,11.16100,3755.700,0.99926,5.338200e+10,1.3867,-0.001878,58.7940,75657.0,187.240,1258.90,0.163340,0.055398,0.020002,7.956400e+08,0.16253,-22.1890,2.0655,0.430880,-10.7410,81606.0,1.1940,1.980400e+14,2017.1,0.46357,1
1,0.129650,0.373480,1763.00,0.72884,0.33247,-1.26310,875.55,554370.0,5.955700e+14,934.430,1.89790,4.905100e+09,0.28471,2.13870,-0.034328,35.223,6.5873,12.7340,29.590,4.1639,31915.00,3.11580,109.430,-0.8522,991.370,4.700100e+09,1.618200e+12,967810.0,1.021500,2.9617,0.810310,415570.0,384720.0,0.002243,6.712900e+15,228.590,33.952,1.4938,2067.50,-0.002343,...,0.067286,5.4044,2.462600e+11,810.3000,860670.0,2165.20,2.711500e+08,10.2320,515.410,262.71,5.6357,0.374220,420.160,116.230,1.26980,0.78619,1207.000,1.00070,2.065000e+13,1.2888,0.248850,9.9857,16323.0,244.460,647.58,0.253820,0.008800,0.018479,1.120600e+08,0.81528,-1.6342,1.5736,-1.071200,11.8320,90114.0,1.1507,4.388000e+16,6638.9,0.28125,0
2,0.120190,0.445210,736.26,0.04615,0.29605,0.31665,2659.50,317140.0,3.977800e+14,131.810,-0.48404,3.800000e+06,0.37173,0.40245,0.015119,1991.600,6.0050,0.9919,11.949,888.0700,887.70,3.52370,108.140,-1.6542,528.140,2.463600e+12,9.005900e+09,7259.3,1.007400,2.6123,1.062500,-208010.0,65708.0,0.001391,8.129800e+13,-84.032,NaN,1.7556,4729.20,0.003527,...,0.064203,3.0197,1.684400e+11,212.8300,7891000.0,277.16,4.047900e+10,1.1170,393.510,8859.10,8.8098,0.405350,393.650,305.140,1.32660,30.85200,2803.700,0.99925,4.722600e+12,1.5003,0.493380,37.0470,139070.0,2464.000,4724.80,0.299160,0.093046,0.018516,6.519300e+07,0.81831,-32.7800,2.1364,-1.931200,-3.2804,37739.0,1.1548,1.718100e+14,5844.0,0.13797,1
3,0.054008,0.395960,996.14,0.85934,0.36678,-0.17060,386.56,325680.0,-3.432200e+13,-26.473,-2.12930,4.675400e+09,0.26659,1.09680,-0.083571,1987.500,6.8989,11.1230,23.164,133.5700,-815.32,2.72950,82.011,3.8374,304.570,-1.315900e+10,-8.324500e+09,2297700.0,0.099522,4.9249,0.659900,48773.0,199560.0,-0.000621,3.993800e+16,1400.400,503.660,1.9131,2811.60,0.002362,...,0.128580,1.4494,2.681000e+11,66.5990,-733900.0,843.42,1.216700e+10,3.6871,243.590,1199.70,4.5092,0.377030,-36.886,11.401,0.97205,23.64200,1882.700,0.99922,1.568000e+13,1.0456,0.548180,27.6650,813040.0,1332.000,359.17,0.026553,0.668620,-0.001260,1.995100e+09,0.86559,-2.4162,1.5199,-0.011633,1.3840,26849.0,1.1490,2.138800e+17,6173.3,0.32910,0
4,0.079947,-0.006919,10574.00,0.34845,0.45008,-1.84200,3027.00,428150.0,9.291500e+11,5999.400,1.87380,1.857300e+09,0.32259,2.21150,-0.107200,661.070,6.5391,6.8495,7.258,468.7400,10316.00,0.79638,111.520,1.4098,429.790,4.121700e+12,-3.945700e+08,7119.4,1.027700,16.1210,0.994510,372800.0,12042.0,-0.000659,1.869200e+14,-129.580,933.150,2.7392,490.84,0.012809,...,0.502290,6.9219,1.487300e+11,9.9686,-35362.0,854.84,2.603900e+10,13.9430,14.552,675.62,6.8424,0.376850,3452.100,35.434,0.94128,0.95142,-56.004,0.99969,2.009700e+13,1.2255,0.713930,14.1820,4362.6,12.007,10653.00,0.165300,0.227090,0.020485,-1.050400e+07,0.25190,-18.6300,3.7387,0.757080,-4.9405,50336.0,1.2488,2.151300e+17,2250.1,0.33796,0


In [39]:
X.pop('id')

0              0
1              1
2              2
3              3
4              4
           ...  
957914    957914
957915    957915
957916    957916
957917    957917
957918    957918
Name: id, Length: 957919, dtype: int64

For multiple missing values, we use Imputation to fill the null values instead of dropping those rows which would lead to loss of information.

In [40]:
imputer =  SimpleImputer(missing_values=np.nan, strategy='median')
X = imputer.fit_transform(X)
X_test = imputer.transform(X_test)

## Important Note
The following cell can be uncommented to run the hyperparameter tunning process which uses hyperopt method

In [43]:
from lightgbm import LGBMClassifier
def optimize(params, x, y):
    #model = ensemble.RandomForestClassifier(**params)
    model = LGBMClassifier(**params)
    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=x, y = y):
        train_idx, val_idx = idx[0], idx[1]
        X_train = x[train_idx]
        y_train = y[train_idx]

        X_valid = x[val_idx]
        y_valid = y[val_idx]

        model.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], 
                  early_stopping_rounds = 300, verbose = False)

        preds = model.predict_proba(X_valid)[:,1]
        fold_acc = metrics.roc_auc_score(y_valid, preds)
        accuracies.append(fold_acc)
    return -1.0*np.mean(accuracies)

## Important note
The following cell can be uncommented to run the hyperparameter tunning process which uses hyperopt method

In [ ]:
param_space = {
        "max_depth": scope.int(hp.quniform("max_depth", 3, 18, 1)),
        "n_estimators": scope.int(hp.quniform("n_estimators", 100, 600, 1)),
        'min_child_weight' : scope.int(hp.quniform('min_child_weight', 0, 10, 1)),
        'reg_alpha' : scope.int(hp.quniform('reg_alpha', 40,180,1)),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
    }
optimization_function = partial(optimize, x=X, y=y)

trials = Trials()

result = fmin(
    
    fn = optimization_function,
    space = param_space,
    algo = tpe.suggest, 
    max_evals = 10,
    trials = trials, 
    )
print(result)

100%|██████████| 10/10 [2:29:42<00:00, 898.25s/it, best loss: -0.8148478271813856]
{'colsample_bytree': 0.6774038062331773, 'max_depth': 7.0, 'min_child_weight': 6.0, 'n_estimators': 499.0, 'reg_alpha': 77.0, 'reg_lambda': 0.8349894643900555}


In [ ]:
best_params = dict(result)
print("Best Parameters: ", best_params)

Best Parameters:  {'colsample_bytree': 0.6774038062331773, 'max_depth': 7.0, 'min_child_weight': 6.0, 'n_estimators': 499.0, 'reg_alpha': 77.0, 'reg_lambda': 0.8349894643900555}


## Performing 10 fold cross validation using LGBMClassifier

In [44]:
from lightgbm import LGBMClassifier
params_lgbm =  {
        'boosting_type': 'gbdt',
        "max_depth": 7,
        "n_estimators": 499,
        'min_child_weight' : 6,
        'reg_alpha' : 77.0,
        'reg_lambda' : 0.834,
        'colsample_bytree' : 0.677,
        'random_state' : 42,
        'n_jobs': -1,
        'metric': 'AUC',
        'verbosity': -1,
    }

folds = model_selection.StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)
y_pred = np.zeros(len(X_test))
for fold, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    
    X = pd.DataFrame(X)
    y = pd.DataFrame(y)
    
    X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]

    model = LGBMClassifier(**params_lgbm)
   
    model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = False, early_stopping_rounds = 300)
    y_pred += model.predict_proba(X_test)[:,1] / folds.n_splits 

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_la

In [46]:
sample_solution = pd.read_csv('sample_solution.csv')
sample_solution['claim'] = y_pred
sample_solution.to_csv('Submission.csv',index = False)


In [48]:
sample = pd.read_csv('Submission.csv')
sample.head()

,id,claim
0,957919,0.589319
1,957920,0.109807
2,957921,0.619079
3,957922,0.127963
4,957923,0.151379
